In [1]:
"""
Created on Thu Apr 16 11:12:44 2017

@author: raysun

Source - https://radimrehurek.com/gensim/tut1.html based on Deerwester et al. 1990 paper, which
         can be found at http://www.cs.bham.ac.uk/~pxt/IDA/lsa_ind.pdf
"""

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from pprint import pprint  # pretty-printer

# Setp 1: From Strings to Vectors
documents = ["Human machine interface for lab ABC computer applications",      # c1
             "A survey of user opinion of computer system response time",      # c2
             "The EPS user interface management system",                       # c3
             "System and human system engineering testing of EPS",             # c4
             "Relation of user perceived response time to error measurement",  # c5
             "The generation of random binary, unordered trees",               # m1
             "The intersection graph of paths in trees",                       # m2
             "Graph minors IV Widths of trees and well quasi ordering",        # m3
             "Graph minors A survey"]                                          # m4

print(documents)


['Human machine interface for lab ABC computer applications', 'A survey of user opinion of computer system response time', 'The EPS user interface management system', 'System and human system engineering testing of EPS', 'Relation of user perceived response time to error measurement', 'The generation of random binary, unordered trees', 'The intersection graph of paths in trees', 'Graph minors IV Widths of trees and well quasi ordering', 'Graph minors A survey']


In [2]:
# Step 2: Tokenize the document. Remove common words and tokenize (bag of words)
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]
print(texts)

[['human', 'machine', 'interface', 'lab', 'abc', 'computer', 'applications'], ['survey', 'user', 'opinion', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'management', 'system'], ['system', 'human', 'system', 'engineering', 'testing', 'eps'], ['relation', 'user', 'perceived', 'response', 'time', 'error', 'measurement'], ['generation', 'random', 'binary,', 'unordered', 'trees'], ['intersection', 'graph', 'paths', 'trees'], ['graph', 'minors', 'iv', 'widths', 'trees', 'well', 'quasi', 'ordering'], ['graph', 'minors', 'survey']]


In [3]:
# Step 3. Remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
         for text in texts]
pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [4]:
# Step 4. Build a dictionary
from gensim import corpora

dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/deerwester.dict')  # store the dictionary, for future reference
print(dictionary)
print(dictionary.token2id)

# Load the dictionary
#dictionary = corpora.Dictionary.load('/tmp/deerwester.dict')
#print(dictionary)

ImportError: libmkl_intel_lp64.so: cannot open shared object file: No such file or directory

In [ ]:
# Step 5. Compute term ocurrences in each doccument
from gensim import corpora

corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('/tmp/deerwester.mm', corpus)  # store to disk, for later use
print(corpus)

#corpus = [ [(0, 1), (1, 1), (2, 1)],                                     # c1
#           [(1, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],             # c2
#           [(0, 1), (6, 1), (7, 1), (8, 1)],                             # c3
#           [(2, 1), (6, 2), (8, 1)],                                     # c4
#           [(3, 1), (4, 1), (7, 1)],                                     # c5
#           [(9, 1)],                                                     # m1
#           [(9, 1), (10, 1)],                                            # m2
#           [(9, 1), (10, 1), (11, 1)],                                   # m3
#           [(5, 1), (10, 1), (11, 1)]]                                   # m4
#print(corpus)

# Load the corpus
corpus = corpora.MmCorpus('/tmp/deerwester.mm') # comes from the first tutorial, "From strings to vectors"
print(corpus)

In [ ]:
# Step 6. Look up dictionary 
new_doc = "Human computer graph"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)  # the word "interaction" does not appear in the dictionary and is ignored

In [ ]:
# Step 7. Compute TF-IDF of the corpus
from gensim import models

tfidf = models.TfidfModel(corpus) # initialize a model
print(tfidf)

# Assign the tfidf to the corpus
corpus_tfidf = tfidf[corpus]

# Print the tfidf of each document
for doc in corpus_tfidf:
    print(doc)

In [ ]:
# Step 8 Compute similarity between a new sentence and the documents in the term-document matrix.
# Define a new sentence
vec = [(1, 1), (2, 1), (10, 1)]
print(tfidf[vec])

from gensim import similarities
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=12)

sims = index[tfidf[vec]]
print(list(enumerate(sims)))

In [ ]:
# Step 9. Perform LSI
from gensim import models

# Run LSI model
lsi = models.LsiModel(corpus=corpus, id2word=dictionary, num_topics=2)
print lsi.print_topics()

# Map tfidf corpus matrix to LSI model
corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
for doc in corpus_lsi: # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
    print(doc) # row: doc; col: 0-1 topic index
    
# Save LSI model    
lsi.save('/tmp/model.lsi') # same for tfidf, lsi, ...
lsi = models.LsiModel.load('/tmp/model.lsi')    

In [ ]:
# Step 10. Compute query similarity

# Note that the class similarities.MatrixSimilarity is only appropriate when the whole set of 
# vectors fits into memory. For example, a corpus of one million documents would require 2GB 
# of RAM in a 256-dimensional LSI space, when used with this class.
#
# Without 2GB of free RAM, you would need to use the similarities.Similarity class. This class 
# operates in fixed memory, by splitting the index across multiple files on disk, called shards. 
# It uses similarities.MatrixSimilarity and similarities.SparseMatrixSimilarity internally, so 
# it is still fast, although slightly more complex.

#index = similarities.docsim.MatrixSimilarity(lsi[corpus],num_features=12,chunksize=256) 
index = similarities.MatrixSimilarity(lsi[corpus],num_features=12,chunksize= 256) # transform corpus to LSI space and index it

index.save('/tmp/deerwester.index')
index = similarities.MatrixSimilarity.load('/tmp/deerwester.index')

doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow] # convert the query to LSI space
print(vec_lsi)

# To obtain similarities of our query document against the nine indexed documents:
sims = index[vec_lsi] # perform a similarity query against the corpus
print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples

# Better way to display the result.
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims) # print sorted (document number, similarity score) 2-tuples


In [ ]:
# Option 1. Perform LDA:
from gensim import corpora, models

# Save the term-document matrix in form of LDA
#corpora.BleiCorpus.serialize('/tmp/corpus.lda-c', corpus)

# Extract 2 LDA topics, using 1 pass and updating once every 1 chunk (1 documents)
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=2, update_every=1, chunksize=1, passes=100)
print lda.print_topics() # first topic 0, and 1

# compare it with LSI
print lsi.print_topics()

# Save LDA output
lda.save('/tmp/model.lda') # same for tfidf, lda, ...
lda = models.LsiModel.load('/tmp/model.lda')

In [ ]:
# Option 2. Compatible with numpy and scipy
import gensim
import numpy as np
number_of_corpus_features = 12
numpy_matrix = np.random.randint(10, size=[5,2])  # random matrix as an example
corpus = gensim.matutils.Dense2Corpus(numpy_matrix)
numpy_matrix = gensim.matutils.corpus2dense(corpus, num_terms=number_of_corpus_features)

import scipy.sparse
scipy_sparse_matrix = scipy.sparse.random(5,2)  # random sparse matrix as example
corpus = gensim.matutils.Sparse2Corpus(scipy_sparse_matrix)
scipy_csc_matrix = gensim.matutils.corpus2csc(corpus)